In [2]:
%pip install "requests==2.28.1"
import sys
from urllib.parse import urljoin
import requests
sys.path.insert(1, '/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/slang-translator/Connection')

from elastic_con import client 
import pandas as pd
import csv
from elasticsearch import Elasticsearch, helpers

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>


In [ ]:
data = pd.read_csv("/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/Urban_dict/urbandict-word-defs.csv", error_bad_lines=False)

In [5]:
# pre-processing the dataset
data


,word_id,word,up_votes,down_votes,author,definition
0,7,Janky,296,255,dc397b2f,Undesirable; less-than optimum.
1,8,slumpin',16,37,dc397b2f,"low down and funky, but [knee deep] enough to ..."
2,9,yayeeyay,19,27,dc397b2f,"affirmation; suggestion of encouragement, appr..."
3,12,hard-core,162,96,d1610749,anything out of our league that can be good or...
4,13,brutal,12,45,40ece1ef,anything that makes you sweat
...,...,...,...,...,...,...
2580582,9043521,Wikipedia,0,0,44335ab6,An online encyclopedia. This site is often not...
2580583,9043525,Giuseppe,1,1,dc81c268,An annoying person but a good friend.
2580584,9043541,pacewhore,0,0,b5e3ec68,Very fit and loving cat with a huge tail
2580585,9043565,FML,1,0,bd352275,Short for: Forgive me Lord


In [6]:
#Keeping the required columns only (word and definition)
df1 = data[['word','definition']]
df1

,word,definition
0,Janky,Undesirable; less-than optimum.
1,slumpin',"low down and funky, but [knee deep] enough to ..."
2,yayeeyay,"affirmation; suggestion of encouragement, appr..."
3,hard-core,anything out of our league that can be good or...
4,brutal,anything that makes you sweat
...,...,...
2580582,Wikipedia,An online encyclopedia. This site is often not...
2580583,Giuseppe,An annoying person but a good friend.
2580584,pacewhore,Very fit and loving cat with a huge tail
2580585,FML,Short for: Forgive me Lord


In [7]:
#checking the null values 
df1[df1.isna().any(axis=1)]

,word,definition
233,craptacular,NaN
477,blows,NaN
1605,wussy,NaN
1678,hinie,NaN
1781,smoove,NaN
...,...,...
1269237,NaN,NaN
1597691,NaN,meaning sick or disgusting
1698995,NaN,The term used quite relatively to shut someone...
1935979,NaN,1. Not-a-Number. These result from operations ...


In [8]:
# removing the null values
df1.dropna(inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
#Verifying the removal
df1[df1.isna().any(axis=1)]

,word,definition


In [10]:
#Checking the shape of the dataset
df1.shape

(2580394, 2)

In [11]:
#Since the size of the dataset is huge, was not able to load the data (even with bulk load and failed iwth connection time-out error) and hence split the dataframe into 2 seperate dataframe and load them under the same index in ES
df_1 = df1.iloc[:1290197,:]
df_2 = df1.iloc[1290197:,:]
print("Shape of new dataframes - {} , {}".format(df_1.shape, df_2.shape))

Shape of new dataframes - (1290197, 2) , (1290197, 2)


In [ ]:
#loading the 2 seperate files 
#df_1.to_csv('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/Urban_dict/urban_dict_updated1.csv')
#df_2.to_csv('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/Urban_dict/urban_dict_updated2.csv')

In [ ]:
with open('/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/Urban_dict/urban_dict_updated1.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(client, reader, index='urban_dict',raise_on_error=False)

In [12]:
# ES search query to fetch all the data 
index_name = "urban_dict"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'urban_dict', '_type': '_doc', '_id': 'hLw8sIUBHHz02O8VY9tZ', '_score': 1.0, '_source': {'word': 'Janky', 'definition': 'Undesirable; less-than optimum.'}}, {'_index': 'urban_dict', '_type': '_doc', '_id': 'hbw8sIUBHHz02O8VY9tZ', '_score': 1.0, '_source': {'word': "slumpin'", 'definition': 'low down and funky, but [knee deep] enough to ride to.'}}, {'_index': 'urban_dict', '_type': '_doc', '_id': 'hrw8sIUBHHz02O8VY9tZ', '_score': 1.0, '_source': {'word': 'yayeeyay', 'definition': 'affirmation; suggestion of encouragement, approval, or interest.'}}, {'_index': 'urban_dict', '_type': '_doc', '_id': 'h7w8sIUBHHz02O8VY9tZ', '_score': 1.0, '_source': {'word': 'hard-core', 'definition': 'anything out of our league that can be good or bad.'}}, {'_index': 'urban_dict', '_type': '_doc', '_id': 'iLw8sIUBHHz02O8VY9tZ', '_score': 1.0, '_source': {'word': 'brutal', 'definition': 'anything that makes you sweat'}}, {'_index': 'urban_dict', '_type': '_doc', '_id': 'ibw8sIUBHHz02O8VY9tZ', '

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
